In [1]:
#%%
import os
import tifffile as tf
import numpy as np
import cv2
from skimage.color import rgb2hed
import dask.array as da
from tqdm import tqdm
from skimage.color import separate_stains, hed_from_rgb
import matplotlib.pyplot as plt
import pyvips
from skimage.exposure import rescale_intensity
import geopandas as gpd 

/rsrch5/home/plm/phacosta/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Helper: Resize image ---
def img_resize(img, scale_factor):
    height = int(np.floor(img.shape[0] * scale_factor))
    width = int(np.floor(img.shape[1] * scale_factor))
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

# --- Your write_ome_tif function ---
def write_ome_tif(filename, image, channel_names,
                  photometric_interp, metadata, subresolutions):
    
    with tf.TiffWriter(filename, bigtiff=True) as tif:
        px_size_x = metadata['PhysicalSizeX']
        px_size_y = metadata['PhysicalSizeY']

        options = dict(
            photometric=photometric_interp,
            tile=(1024, 1024),
            maxworkers=4,
            compression='lzw',
            resolutionunit='CENTIMETER',
        )

        print("Writing pyramid level 0")
        tif.write(
            image,
            subifds=subresolutions,
            resolution=(1e4 / px_size_x, 1e4 / px_size_y),
            metadata=metadata,
            **options,
        )

        scale = 1
        for i in range(subresolutions):
            scale *= 0.5
            if photometric_interp == 'minisblack':
                if image.shape[0] < image.shape[-1]:  # (C, H, W) -> (H, W, C)
                    image = np.moveaxis(image, 0, -1)
                    image = img_resize(image, 0.5)
                    image = np.moveaxis(image, -1, 0)
                else:
                    image = img_resize(image, 0.5)
            else:
                image = img_resize(image, 0.5)

            print(f"Writing pyramid level {i+1}")
            tif.write(
                image,
                subfiletype=1,
                resolution=(1e4 / scale / px_size_x, 1e4 / scale / px_size_y),
                **options
            )
    print("Saved:", filename)

In [3]:
# %% Helper Function (from your example) for pyramid generation

# core_name = "A-2"
file_num = 0 # Selects the first H&E file from the dictionary
skip_existing = False 

root_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration"
data_dict = {
    "output-XETG00522__0066398__Region_1__20250515__183305": "Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif",
    "output-XETG00522__0066402__Region_1__20250515__183305": "Xenium H&E PCF TMA 5-28-2025_matching_orientation.ome.tif"
}

xenium_folder = list(data_dict.keys())[file_num]
slide_name = data_dict[xenium_folder]

he_annot = os.path.join(root_dir, slide_name.replace(".ome.tif", "_annot.geojson"))
                          
# load core polygons ---------------------------------------
gdf = gpd.read_file(he_annot)
# gdf = gdf[gdf['isMissing'] == False].reset_index(drop=True)

for idx, row in gdf.iterrows():
    core_name = row.get('name', f'core_{idx + 1}')
    ome_tif_path = os.path.join(root_dir, "HnE", slide_name, f"he_{core_name}.ome.tif")
    
    # Define the output path for the new HED image
    output_filename = os.path.join(ome_tif_path.rsplit("/", 1)[0], ome_tif_path.rsplit("/", 1)[1].replace(".ome.tif", "_HED.ome.tif"))
    if skip_existing and os.path.exists(output_filename):
        print(f"{output_filename} exists. Skipping")
        continue
    
    assert os.path.exists(ome_tif_path)
    print(f"Input H&E file: {ome_tif_path}")
    print(f"Output HED file: {output_filename}")
    
    # --- Hematoxylin Extraction Settings ---
    tile_size = 1024
    output_path = output_filename
    temp_path = os.path.join(os.path.dirname(output_filename), "hematoxylin_temp.dat")
    
    save_as_uint8 = True  # Use uint8 for QuPath compatibility
    subres_levels = 7     # Number of pyramid levels
    
    # --- Metadata from original image ---
    with tf.TiffFile(ome_tif_path) as tif:
        if not tif.is_ome:
            raise RuntimeError("Input is not OME‑TIFF.")
        
        ome_dict = tf.xml2dict(tif.ome_metadata)
        pixels = ome_dict['OME']['Image']['Pixels']
        
        px_x = float(pixels['PhysicalSizeX'])
        px_y = float(pixels['PhysicalSizeY'])
        unit = pixels['PhysicalSizeXUnit']
        
        try:
            channel_names = [ch['Name'] for ch in pixels['Channel']]
        except (KeyError, TypeError):
            channel_names = None
        
        photometric = 'rgb' if tif.pages[0].samplesperpixel == 3 else 'minisblack'
    
    # For the hematoxylin image (grayscale), override photometric and channel name
    metadata = {
        'PhysicalSizeX': px_x,
        'PhysicalSizeXUnit': unit,
        'PhysicalSizeY': px_y,
        'PhysicalSizeYUnit': unit,
        'Channel': {'Name': ["hematoxylin"]},
    }
    
    # --- Load full-resolution image ---
    vips_img = pyvips.Image.new_from_file(ome_tif_path, access="sequential")
    H, W = vips_img.height, vips_img.width
    hema_memmap = np.memmap(temp_path, dtype=np.float32, mode='w+', shape=(H, W))
    
    # --- Tile-wise deconvolution ---
    for y in tqdm(range(0, H, tile_size), desc="Extracting hematoxylin"):
        for x in range(0, W, tile_size):
            h = min(tile_size, H - y)
            w = min(tile_size, W - x)
    
            tile_vips = vips_img.crop(x, y, w, h)
            tile_np = np.ndarray(
                buffer=tile_vips.write_to_memory(),
                dtype=np.uint8,
                shape=(tile_vips.height, tile_vips.width, tile_vips.bands),
            ).astype(np.float32) / 255.0
    
            hema_channel = separate_stains(tile_np, hed_from_rgb)[:, :, 0]
            hema_memmap[y:y+h, x:x+w] = hema_channel
    
    # --- Save output using your write_ome_tif ---
    hema_memmap = np.memmap(temp_path, dtype=np.float32, mode='r', shape=(H, W))
    
    if save_as_uint8:
        hema_uint8 = rescale_intensity(hema_memmap, out_range=(0, 255)).astype(np.uint8)
        write_ome_tif(
            filename=output_path,
            image=hema_uint8,
            channel_names=["hematoxylin"],
            photometric_interp="minisblack",
            metadata=metadata,
            subresolutions=subres_levels
        )
    else:
        write_ome_tif(
            filename=output_path,
            image=hema_memmap,
            channel_names=["hematoxylin"],
            photometric_interp="minisblack",
            metadata=metadata,
            subresolutions=subres_levels
        )
    
    # --- Cleanup ---
    os.remove(temp_path)

Input H&E file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-4.ome.tif
Output HED file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-4_HED.ome.tif


Extracting hematoxylin: 100%|██████████| 106/106 [11:02<00:00,  6.25s/it]


Writing pyramid level 0
Writing pyramid level 1
Writing pyramid level 2
Writing pyramid level 3
Writing pyramid level 4
Writing pyramid level 5
Writing pyramid level 6
Writing pyramid level 7
Saved: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-4_HED.ome.tif
Input H&E file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-6.ome.tif
Output HED file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-6_HED.ome.tif


Extracting hematoxylin: 100%|██████████| 106/106 [11:02<00:00,  6.25s/it]


Writing pyramid level 0
Writing pyramid level 1
Writing pyramid level 2
Writing pyramid level 3
Writing pyramid level 4
Writing pyramid level 5
Writing pyramid level 6
Writing pyramid level 7
Saved: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-6_HED.ome.tif
Input H&E file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_C-3.ome.tif
Output HED file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_C-3_HED.ome.tif


...

Writing pyramid level 0
Writing pyramid level 1
Writing pyramid level 2
Writing pyramid level 3
Writing pyramid level 4
Writing pyramid level 5
Writing pyramid level 6
Writing pyramid level 7
Saved: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_D-4_HED.ome.tif
Input H&E file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_B-5.ome.tif
Output HED file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_B-5_HED.ome.tif


Extracting hematoxylin: 100%|██████████| 106/106 [10:35<00:00,  5.99s/it]


Writing pyramid level 0
Writing pyramid level 1
Writing pyramid level 2
Writing pyramid level 3
Writing pyramid level 4
Writing pyramid level 5
Writing pyramid level 6
Writing pyramid level 7
Saved: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_B-5_HED.ome.tif
Input H&E file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-5.ome.tif
Output HED file: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-5_HED.ome.tif


Extracting hematoxylin: 100%|██████████| 106/106 [10:34<00:00,  5.99s/it]


Writing pyramid level 0
Writing pyramid level 1
Writing pyramid level 2
Writing pyramid level 3
Writing pyramid level 4
Writing pyramid level 5
Writing pyramid level 6
Writing pyramid level 7
Saved: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/HnE/Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif/he_A-5_HED.ome.tif
